Puedes utilizar estos entornos para ejecutar el código (si lo haces así, tienes que subir los datos)

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/15_processing_sequences_using_rnns_and_cnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml3/blob/main/15_processing_sequences_using_rnns_and_cnns.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

## REDES RECURRENTES (aplicado a series temporales)

Adaptado de [Hands on Machine Learning for Python](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/ch15.html#cnn_chapter), utilizando lo aprendido en las sesiones de series temporales

Requisitos y recomendaciones:  
* Python 3.7 o superior
* Tensorflow 2.8 o superior
* Es preferible utilizar un entorno con GPU (si se quiere probar la parte LSTM y GRU mejor con Nvida, ya que Keras emplea cuRNN que es una mejora basada en CUDA para Nvidia), por ejemplo Colab de Google

In [ ]:
# Instalamos pmdarima: librería para modelos ARIMA automáticos
# Esta librería nos permite crear modelos ARIMA/SARIMA de forma automática
# seleccionando los mejores hiperparámetros mediante búsqueda
!pip install pmdarima

In [ ]:
# IMPORTANTE: pmdarima requiere específicamente numpy 1.26.4 para funcionar correctamente
# Las versiones más recientes pueden causar incompatibilidades

In [ ]:
# Instalamos la versión específica de numpy compatible con pmdarima
!pip install numpy==1.26.4

In [ ]:
# IMPORTANTE: Después de cambiar la versión de numpy, 
# debes reiniciar la sesión/kernel para que los cambios surtan efecto
# En Google Colab o Jupyter, usa: Runtime > Restart runtime

In [ ]:
# Importamos numpy y verificamos que la versión instalada sea la correcta
import numpy as np
np.__version__  # Debe mostrar 1.26.4

In [ ]:
# ============================================================================
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
# ============================================================================

import numpy as np          # Operaciones numéricas y arrays
import pandas as pd         # Manipulación de datos tabulares y series temporales
import sys                  # Información del sistema
import tensorflow as tf     # Framework de Deep Learning
import warnings
warnings.filterwarnings('ignore')  # Ocultamos warnings para claridad en la salida

# Librerías para modelos ARIMA tradicionales
from pmdarima.arima import ARIMA, auto_arima

# Métricas de evaluación
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

# Análisis de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose  # Descomposición de series
from statsmodels.tsa.stattools import adfuller           # Test de estacionariedad

# Verificamos que tenemos TensorFlow 2.8 o superior
from packaging import version
assert version.parse(tf.__version__) >= version.parse("2.8.0")

Algunas preconfiguraciones para hacer más "visibles" los gráficos

In [ ]:
# ============================================================================
# CONFIGURACIÓN DE MATPLOTLIB PARA GRÁFICOS MÁS LEGIBLES
# ============================================================================

import matplotlib.pyplot as plt

# Configuramos el tamaño de fuentes para que los gráficos sean más visibles
# Esto es especialmente útil en presentaciones o clases
plt.rc('font', size=14)           # Tamaño general de fuente
plt.rc('axes', labelsize=14, titlesize=14)  # Tamaño de etiquetas y títulos
plt.rc('legend', fontsize=14)     # Tamaño de la leyenda
plt.rc('xtick', labelsize=10)     # Tamaño de etiquetas eje X
plt.rc('ytick', labelsize=10)     # Tamaño de etiquetas eje Y

### El problema: Predecir el uso del transporte público en Chicago

Como ya hicimos al ver las series temporales con modelos tradicionales, vamos a utilizar los datos diarios de utilización de autobuses y tren de la ciudad de Chicago. Te recuerdo que están extraídos de su portal de datos públicos, y te recomiendo que te des una vuelta por él. Chicago es una de las ciudades más Smart del mundo y llevan recogiendo datos de diversos temas desde hace mucho tiempo... [Chicago's Data Portal](https://data.cityofchicago.org/)


In [ ]:
# Creamos un directorio para almacenar los datos
# El signo ! ejecuta comandos del sistema operativo desde el notebook
!mkdir data

In [ ]:
# Movemos el archivo CSV de datos al directorio 'data/'
# Este archivo contiene los datos de uso diario del transporte público en Chicago
!mv CTA_-_Ridership_-_Daily_Boarding_Totals.csv data/

Cargemos y preparemos los datos. Recuerda en series temporales conviene convertir a datatime las fechas y usarlas como índices.

In [ ]:
# ============================================================================
# CARGA Y PREPARACIÓN DE LOS DATOS
# ============================================================================

import pandas as pd

DATA_PATH = "./data/CTA_-_Ridership_-_Daily_Boarding_Totals.csv"

# Cargamos el CSV y parseamos automáticamente la columna de fechas
df = pd.read_csv(DATA_PATH, parse_dates=["service_date"])

# Renombramos las columnas para que sean más descriptivas y fáciles de usar
df.columns = ["date", "day_type", "bus", "rail", "total"]

# Convertimos la columna 'date' a formato datetime (por si acaso)
df["date"] = pd.to_datetime(df["date"])

# IMPORTANTE: En series temporales, ordenamos por fecha y la usamos como índice
# Esto facilita el slicing temporal (ej: df["2019-01":"2019-06"])
df = df.sort_values("date").set_index("date")

# ============================================================================
# LIMPIEZA DE DATOS
# ============================================================================

# Eliminamos la columna 'total' porque es redundante (bus + rail)
df = df.drop("total", axis=1)

# Eliminamos filas duplicadas que existen en el dataset original
df = df.drop_duplicates()

Como en cualquier otro dataset echamos un vistazo, pero además siendo una serie recuerda que es importante descomponerla y analizar su estacionariedad.

In [ ]:
# Visualizamos las primeras filas del dataframe para entender su estructura
# Columnas: day_type (tipo de día), bus (viajes en bus), rail (viajes en tren/metro)
df.head()

In [ ]:
# ============================================================================
# VISUALIZACIÓN DE LA SERIE TEMPORAL
# ============================================================================

# Graficamos el uso de transporte desde 2010 hasta enero 2020 (pre-COVID)
# Al tener 'date' como índice, podemos hacer slicing temporal directamente
df["2010-07-11":"2020-01-31"].plot(
    grid=True,          # Añadimos una cuadrícula para facilitar la lectura
    marker=".",         # Marcamos cada punto de datos
    figsize=(15, 5)     # Tamaño del gráfico (ancho, alto)
)
plt.tight_layout()      # Ajusta automáticamente el espaciado del gráfico

Como recordarás de la unidad de series temporales... ok, vale, no lo recuerdas, pero te lo recuerdo yo... estas series tenían tendencia y dos tipos de estacionalidad (semanal y anual)

Para ver la estacionalidad semanal, pintábamos la serie y su desplazada una semana:

In [ ]:
# ============================================================================
# ANÁLISIS DE ESTACIONALIDAD SEMANAL
# ============================================================================

# Definimos el periodo que queremos visualizar
comienzo = "2019-03"
fin = "2019-05"

fig, axs = plt.subplots(1, 1, figsize=(8, 5))

# Graficamos la serie original
df[comienzo:fin].plot(ax=axs, legend=False, marker=".")

# Graficamos la serie desplazada 7 días (una semana)
# shift(7) mueve todos los valores 7 posiciones hacia adelante
# Si las curvas se superponen, hay estacionalidad semanal
df[comienzo:fin].shift(7).plot(
    ax=axs, 
    grid=True, 
    legend=False, 
    linestyle="--"  # Línea discontinua para distinguirla
)

# INTERPRETACIÓN: Si ambas líneas coinciden, significa que el patrón
# se repite cada 7 días (lunes con lunes, martes con martes, etc.)

Se ve claramente como se superponen casi perfectamente, lo que además nos invitaba a generar un modelo "naive" como baseline. Pero en este caso haremos uso de un modelo SARIMA como baseline. Antes descompogamos la serie mensualizandola (calculando medias por meses) para ver la estacionalidad anual:

(Recordemos que podíamos hacer descomposicion multiplicativa o aditiva)

In [ ]:
# ============================================================================
# DESCOMPOSICIÓN DE LA SERIE TEMPORAL PARA VER ESTACIONALIDAD ANUAL
# ============================================================================

# Para ver la estacionalidad anual, agregamos los datos por mes (promedio mensual)
# resample("M") agrupa por mes, mean() calcula el promedio
df_mensualizada = df.resample("M").mean(numeric_only=True)

# Descomponemos la serie en: Tendencia + Estacionalidad + Residuos
# model='additive': asume que Serie = Tendencia + Estacionalidad + Ruido
# (alternativa: 'multiplicative' donde Serie = Tendencia * Estacionalidad * Ruido)
result_add = seasonal_decompose(
    df_mensualizada["bus"][:"2019-12-31"],  # Usamos datos hasta 2019
    model='additive',
    extrapolate_trend='freq'  # Extrapola la tendencia en los extremos
)

# Configuramos el tamaño de la figura
plt.rcParams.update({'figure.figsize': (6, 6)})

# Visualizamos los 4 componentes: Original, Tendencia, Estacionalidad, Residuos
result_add.plot()

# INTERPRETACIÓN:
# - Observed: Serie original
# - Trend: Tendencia a largo plazo (crecimiento/decrecimiento general)
# - Seasonal: Patrón que se repite anualmente
# - Residual: Lo que queda después de quitar tendencia y estacionalidad

Se puede observar la repetición del patrón anual.

Además también existe una tendencia clara. En definitiva no son series estacionarias, aunque por teminar de recordar vamos a hacer el test de Dickey-Fuller aumentado o ADF test:

In [ ]:
# ============================================================================
# TEST DE ESTACIONARIEDAD: AUGMENTED DICKEY-FULLER (ADF)
# ============================================================================

from statsmodels.tsa.stattools import adfuller

# El test ADF comprueba si una serie es estacionaria
# H0 (hipótesis nula): La serie NO es estacionaria
# H1 (hipótesis alternativa): La serie SÍ es estacionaria

result = adfuller(df['bus'].values)

# result[1] es el p-value
# Si p-value < 0.05 → Rechazamos H0 → La serie ES estacionaria
# Si p-value >= 0.05 → No podemos rechazar H0 → La serie NO es estacionaria
result[1]

El p-value es mayor de 0.05 así que no podemos rechazar la hipótesis nula de no estacionariedad (como ya esperábamos)

En definitiva, si estuvieramos intentando crear un modelo predictivo de la serie temporal, utilizaríamos un SARIMA con diferenciación (d distinto de 0) y con estacionalidad.

De hecho, empleemos como baseline el SARIMA que vimos en la unidad de series temporales, primero prediciendo a 14 días vista:

In [ ]:
# ============================================================================
# MODELO BASELINE: SARIMA (Seasonal ARIMA)
# ============================================================================

# Definimos el periodo de entrenamiento
origin, today = "2019-01-01", "2019-05-31"

# Extraemos la serie de datos de 'rail' (uso del tren/metro)
# asfreq("D") asegura que la frecuencia es diaria, rellenando si faltan datos
rail_series = df.loc[origin:today]["rail"].asfreq("D")

# Datos de validación: junio de 2019
rail_series_valid = df.loc["2019-06":"2019-06"]["rail"].asfreq("D")

# ============================================================================
# CONFIGURACIÓN DEL MODELO SARIMA
# ============================================================================
# ARIMA(p, d, q) + Componente Estacional(P, D, Q, s)
# 
# Parámetros no estacionales:
#   p=1: orden autoregresivo (usa 1 valor pasado)
#   d=0: orden de diferenciación (0 = no diferenciamos)
#   q=1: orden de media móvil
#
# Parámetros estacionales:
#   P=0: componente autoregresivo estacional
#   D=1: diferenciación estacional (elimina estacionalidad)
#   Q=1: componente de media móvil estacional
#   s=7: periodo estacional (7 días = 1 semana)

model = ARIMA(
    order=(1, 0, 1),              # (p, d, q)
    seasonal_order=(0, 1, 1, 7)   # (P, D, Q, s)
)

# Entrenamos el modelo con los datos históricos
model = model.fit(rail_series)

# Predicción de 1 día adelante
y_pred = model.predict(1)  # Devuelve aproximadamente 427,758.6 viajes

In [ ]:
# ============================================================================
# EVALUACIÓN DEL MODELO SARIMA: Predicción a 14 días
# ============================================================================

# Tomamos los primeros 14 días de junio como valores reales
y_valid = rail_series_valid.iloc[:14]

# Predecimos los próximos 14 días (2 semanas)
y_pred = model.predict(14)

# Graficamos predicción vs realidad
plt.plot(y_valid, label="Real")
plt.plot(y_pred, color='red', label="SARIMA")
plt.xticks(rotation=45)  # Rotamos etiquetas del eje X para mejor legibilidad
plt.legend()

# NOTA: Esta será nuestra predicción de referencia (baseline) para comparar
# con los modelos de Deep Learning que desarrollaremos después

In [ ]:
# ============================================================================
# MÉTRICAS DE EVALUACIÓN DEL BASELINE
# ============================================================================

# RMSE (Root Mean Squared Error): Error cuadrático medio
# Penaliza más los errores grandes. Mismo orden de magnitud que los datos.
print("RMSE:", np.sqrt(mean_squared_error(y_valid, y_pred)))

# MAPE (Mean Absolute Percentage Error): Error porcentual absoluto medio
# Expresa el error como porcentaje, fácil de interpretar
# Ej: 5% significa que en promedio nos equivocamos un 5%
print("MAPE:", mean_absolute_percentage_error(y_valid, y_pred) * 100)

# OBJETIVO: Nuestros modelos de RNN deben superar estas métricas

Muy bien ahí tenemos nuestro baseline, ahora veamos que tal lo hacemos usando redes neuronales

***

### Con MLPs o Redes densas (DNN, deep neural networks)

Las redes recurrentes además de su complejidad para "visualizarlas" mentalmente y la propia de tener que configurarlas como cualquier otra capa de DL, añaden la farragosa tarea de preparar los dataset de entrada... Tal como vimos al hablar de entrenamiento en las sesiones teóricas.

Entre otras cosas porque ahora la red espera batches de secuencias con multiples posibilidades en los targets...


Por ejemplo, si tuvieramos una serie temporal de ingresos de una empresa con valores diarios tal como:

[12500, 3500, 1234, 111000, 2345, 8889, 12567]

Y quisieramos predecir el día siguiente con los tres días anteriores tendríamos que construir el siguiente dataset:

[12500,3500,1234], target: [111000]  
[3500,1234,111000], target: [2345]  
[1234,111000,2345], target: [8889]  
[111000,2345,8889], target: [12567]  

Donde ahora cada fila es una instancia y una secuencia con su target

Para poder preparar los batches y el dataset de entrenamiento a partir del dataframe con los datos, Keras nos da ciertas "facilidades"

In [ ]:
# ============================================================================
# PREPARACIÓN DE DATASETS PARA REDES NEURONALES
# ============================================================================

# EJEMPLO SIMPLE: Cómo funciona timeseries_dataset_from_array
# Esta función crea secuencias deslizantes (sliding windows) automáticamente

my_series = [0, 1, 2, 3, 4, 5]

# Creamos un dataset donde:
# - Cada secuencia tiene 3 valores consecutivos
# - El target es el valor 3 pasos adelante
# - Agrupamos en batches de 2 secuencias
my_dataset = tf.keras.utils.timeseries_dataset_from_array(
    my_series,
    targets=my_series[3:],  # Los targets están 3 pasos en el futuro
    sequence_length=3,       # Longitud de cada secuencia de entrada
    batch_size=2,           # Número de secuencias por batch
    shuffle=False           # No mezclamos (para ver el orden claramente)
)

# Visualizamos los batches creados
list(my_dataset)

# RESULTADO:
# Batch 1: secuencias [0,1,2] y [1,2,3] con targets [3] y [4]
# Batch 2: secuencia [2,3,4] con target [5]

Ha creado dos batches, el primero con las dos posibles primeras secuencias posibles de 3 intervalos [0,1,2] y [1,2,3] y el segundo con una sóla secuencia porque no tiene más datos para generarla: [2,3,4]

In [ ]:
# ============================================================================
# EJEMPLO CON DATOS REALES: Predicción de ingresos empresariales
# ============================================================================

my_series = [12500, 3500, 1234, 111000, 2345, 8889, 12567]

my_dataset = tf.keras.utils.timeseries_dataset_from_array(
    my_series,
    targets=my_series[3:],  # Queremos predecir 3 días adelante
    sequence_length=3,       # Usamos 3 días de historia
    batch_size=2,
    shuffle=True  # shuffle=True mezcla las secuencias (útil para entrenamiento)
)

list(my_dataset)

# INTERPRETACIÓN:
# Cada secuencia de 3 días consecutivos predice el día que está 3 pasos adelante
# Ejemplo: [12500, 3500, 1234] predice [111000]

Hacemos tres datasets: entrenamiento, validación y test y aplicamos "normalización" casera, aunque al ser una serie univariante (sólo vamos a usar por ahora una única característica) no debería afectarle

In [ ]:
# Verificamos el valor máximo de la serie 'rail' para decidir la normalización
df["rail"].max()

In [ ]:
# Verificamos el valor máximo de la serie 'bus'
df["bus"].max()

In [ ]:
# ============================================================================
# DIVISIÓN Y NORMALIZACIÓN DE LOS DATOS
# ============================================================================

# Dividimos en train/validation/test y normalizamos dividiendo por 1 millón
# Esto ayuda a que la red neuronal converja más rápido

# TRAIN: Enero 2016 - Mayo 2019 (período de entrenamiento)
rail_train = df["rail"]["2016-01":"2019-05"] / 1e6

# VALIDATION: Necesitamos 56 días antes de junio para poder crear secuencias
# Abril 6 - Junio 2019 (los últimos 56 días servirán como contexto)
rail_valid = df["rail"]["2019-04-06":"2019-06"] / 1e6

# TEST: Julio 2019 en adelante
rail_test = df["rail"]["2019-07":] / 1e6

# ¿Por qué normalizar? 
# Los valores originales están en cientos de miles (ej: 500,000 viajes)
# Dividir por 1e6 los convierte a decimales (0.5), más fáciles de procesar

De los dataframes pasamos a los datasets preparados para entrenar las capas recurrentes usando el comentado timeseries_dataset_from_array y construyendo el dataset de entrada para una predicción al día siguiente después de 56 días (es decir 8 semanas) (esta es una selección arbitraría y podríamos haber empleado otro criterio, pero son más o menos dos meses).

In [ ]:
# ============================================================================
# CREACIÓN DE DATASETS PARA ENTRENAMIENTO Y VALIDACIÓN
# ============================================================================

seq_length = 56  # Usamos 56 días (8 semanas) para predecir el día siguiente

tf.random.set_seed(42)  # Para reproducibilidad de resultados

# DATASET DE ENTRENAMIENTO
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],  # Target: el día después de cada secuencia
    sequence_length=seq_length,        # Ventana de 56 días
    batch_size=32,                     # 32 secuencias por batch
    shuffle=True,                      # Mezclamos para mejor generalización
    seed=42
)
# IMPORTANTE: shuffle=True significa que NO tomamos secuencias consecutivas
# sino que las mezclamos aleatoriamente. Esto evita que la red se sobreajuste
# a patrones temporales específicos del orden de entrenamiento

# DATASET DE VALIDACIÓN
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)
# NOTA: En validación NO mezclamos (shuffle=False por defecto)
# Queremos evaluar en orden cronológico: días 1-56 predicen día 57, etc.

In [ ]:
# Visualizamos un batch aleatorio (el número 14) del dataset de entrenamiento
# Esto nos ayuda a entender la estructura de los datos
list(train_ds)[14]

# ESTRUCTURA:
# (array de secuencias, array de targets)
# Cada secuencia tiene 56 valores, cada target es 1 valor

In [ ]:
# Visualizamos el primer batch del dataset de validación
list(valid_ds)[0]

In [ ]:
# Verificamos los primeros 57 días de validación para entender qué predecimos
rail_valid.iloc[:57]

***


### Construyendo el modelo

Vamos a construir un modelo supersimple sin capa oculta, sólo una neurona sin función de activación (es decir una capa de salida de un modelo de regresión):

In [ ]:
# ============================================================================
# PRIMER MODELO: REGRESIÓN LINEAL SIMPLE (RED DENSA)
# ============================================================================

tf.random.set_seed(42)  # Reproducibilidad

# Modelo MUY simple: Solo una capa densa con 1 neurona
# Es equivalente a una regresión lineal sobre los 56 días anteriores
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[seq_length])
    # input_shape=[56]: recibe 56 valores (los 56 días)
    # 1 neurona: devuelve 1 valor (la predicción del día siguiente)
    # Sin función de activación = regresión lineal
])

# Configuración del entrenamiento
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",           # Monitoreamos el error absoluto en validación
    patience=50,                 # Esperamos 50 épocas sin mejora antes de parar
    restore_best_weights=True    # Al terminar, restauramos los mejores pesos
)

# Optimizador SGD (Stochastic Gradient Descent) con momentum
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

# Compilación del modelo
model.compile(
    loss=tf.keras.losses.Huber(),  # Función de pérdida robusta a outliers
    optimizer=opt,
    metrics=["mae"]  # Mean Absolute Error como métrica de seguimiento
)

# ¿Por qué Huber Loss?
# Es más robusta que MSE cuando hay valores atípicos (outliers)
# Para errores pequeños se comporta como MSE (cuadrática)
# Para errores grandes se comporta como MAE (lineal)

Antes de entrenar, entendamos que estamos haciendo: Una regresión lineal de los 56 días anteriores al que quiero predecir, es parecido a hacer un ARIMA con p = 56, d = 0, q = 0

In [ ]:
# Entrenamos el modelo
# Máximo 500 épocas, pero el early stopping probablemente parará antes
history = model.fit(
    train_ds, 
    validation_data=valid_ds, 
    epochs=500,
    callbacks=[early_stopping_cb]
)

In [ ]:
# Evaluamos el modelo en el conjunto de validación
valid_loss, valid_mae = model.evaluate(valid_ds)

# Desnormalizamos el error multiplicando por 1e6
# Recordemos que dividimos por 1e6, ahora multiplicamos para volver a escala original
valid_mae * 1e6

# IMPORTANTE: Este MAE es para predicciones de 1 día adelante
# No es directamente comparable con el SARIMA que predecía 14 días

Hmm, parece mucho mejor... ?verdad? Pues no exactamente, porque si te fijas esta validación es día a día (es decir no prediciendo con los datos hasta hoy los próximos 14 días, sino prediciendo cada uno de los próximos 30 días, los de junio de 2019 que es el validation set, con los 56 días anteriores a cada día). En sesiones posteriores veremos la comparación correcta con el baseline de SARIMA.

Por otro lado y aunque lo parezca, no hemos hecho uso del orden de los datos, es decir tal como hemos entrenado podríamos haber desordenado las secuencias internamente (cambiando el orden de la misma forma en todas, por ejemplo intercambiando el día 23 con el 47 y el 12 con el 2, etc) que hubiera salido el mismo resultado. No estamos teniendo en cuenta el orden y para eso introduciremos las redes recurrentes.

Pero antes de terminar un pequeño inciso para introducir la función de pérdida Huber (sí, si te has fijado bien es la que hemos usado):

### Función de pérdida Huber

La función de pérdida de Huber es una combinación de error cuadrático medio (MSE, por sus siglas en inglés) y error absoluto medio (MAE, por sus siglas en inglés), diseñada para ser robusta a outliers en los datos. La principal diferencia entre la pérdida de Huber y la MSE radica en cómo tratan los errores grandes:

**MSE (Mean Squared Error)**: Calcula el promedio de los cuadrados de los errores entre los valores predichos y los reales. Tiende a penalizar mucho los errores grandes, lo que puede llevar a una sensibilidad excesiva a outliers en el conjunto de datos.  

**Pérdida de Huber**: Es menos sensible a los outliers que MSE. Para errores pequeños, funciona como MSE, y para errores grandes, se comporta como MAE, haciendo que la pérdida sea lineal en vez de cuadrática con respecto a la diferencia entre el valor predicho y el real. Esto se logra mediante un (hiper)parámetro delta (δ), que define el umbral entre tratar un error como grande o pequeño.

#### ¿Cuándo usar la pérdida de Huber frente a la MSE?

La pérdida de Huber se prefiere sobre MSE en situaciones donde hay una expectativa de outliers en los datos, o cuando no se desea que los errores grandes dominen la función de pérdida. Es decir, si tu conjunto de datos incluye valores anómalos que podrían afectar negativamente el proceso de entrenamiento del modelo con MSE, la pérdida de Huber puede ofrecer un enfoque más equilibrado y robusto.

Por otro lado, MSE puede ser preferible en situaciones donde todos los errores se consideran igualmente importantes, y se desea penalizar más fuertemente los errores grandes para enfocarse en minimizar estos errores específicos durante el entrenamiento.

En nuestro caso podríamos haber empleado una MSE, pero como hay variaciones fuertes entre los años usamos una Huber, además por defecto delta (δ) vale 1, al haber normalizado al millón lo que estamos diciendo es que penalice más los errores superiores al millón (considerándolos como outliers y penalizándolos de forma cuadrática) y los errores inferiores al millón los considere equilibrados (pesándolos en términos absolutos). Si tu rango de valores es diferente tendrás que ajustar delta de forma adecuada.

***

## Using a Simple RNN

Ahora vamos a emplear una única capa con una única celda o neurona recurrente sencilla (el hidden_stat(t) = output(t-1) y hidden_state(0) = 0)
Es interesante darse cuenta de que la función de activación no es una relu es una tanh.

In [ ]:
# ============================================================================
# PRIMERA RED RECURRENTE: SimpleRNN con 1 neurona
# ============================================================================

tf.random.set_seed(42)  # Reproducibilidad

model = tf.keras.Sequential([
    # SimpleRNN: Capa recurrente básica
    # 1 neurona recurrente que procesa la secuencia paso a paso
    # input_shape=[None, 1]: 
    #   - None: secuencias de longitud variable
    #   - 1: una característica por paso temporal (solo el valor de 'rail')
    tf.keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

# DIFERENCIA CLAVE con la red densa anterior:
# - Red densa: trata los 56 días como 56 características independientes
# - SimpleRNN: procesa los 56 días SECUENCIALMENTE, manteniendo memoria
#
# La RNN tiene "memoria" porque el estado oculto de t-1 influye en t
# Fórmula: h(t) = tanh(W_x * x(t) + W_h * h(t-1) + b)
# 
# NOTA: La función de activación por defecto es tanh (no ReLU)

In [ ]:
# ============================================================================
# FUNCIÓN AUXILIAR: Entrenar y Evaluar modelos
# ============================================================================

def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500, patience=None):
    """
    Función reutilizable para entrenar y evaluar modelos de series temporales
    
    Args:
        model: Modelo de Keras a entrenar
        train_set: Dataset de entrenamiento
        valid_set: Dataset de validación
        learning_rate: Tasa de aprendizaje para SGD
        epochs: Número máximo de épocas (default: 500)
        patience: Épocas sin mejora antes de parar (default: 10% de epochs)
    
    Returns:
        MAE en validación (desnormalizado, en escala original)
    """
    # Si no especifican patience, usamos el 10% de las épocas
    patience = int(epochs // 10) if patience == None else patience
    
    # Early stopping para evitar sobreajuste
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", 
        patience=patience, 
        restore_best_weights=True
    )
    
    # Optimizador SGD con momentum (ayuda a escapar de mínimos locales)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    
    # Compilamos el modelo
    model.compile(
        loss=tf.keras.losses.Huber(),  # Robusta a outliers
        optimizer=opt, 
        metrics=["mae"]
    )
    
    # Entrenamos
    history = model.fit(
        train_set, 
        validation_data=valid_set, 
        epochs=epochs,
        callbacks=[early_stopping_cb]
    )
    
    # Evaluamos y desnormalizamos
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6  # Volvemos a escala original

In [ ]:
# Entrenamos y evaluamos la SimpleRNN de 1 neurona
fit_and_evaluate(model, train_ds, valid_ds, learning_rate=0.02)

Y ahora ya vamos a emplear una capa con 32 neurona y luego una capa densa que nos de la regresión, sin función de activación

In [ ]:
# ============================================================================
# RED RECURRENTE MEJORADA: SimpleRNN con 32 neuronas + Capa Densa
# ============================================================================

tf.random.set_seed(42)

univar_model = tf.keras.Sequential([
    # Capa recurrente con 32 neuronas
    # Esto permite capturar 32 "características temporales" diferentes
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 1]),
    
    # Capa densa de salida
    # Combina las 32 salidas de la RNN en una única predicción
    tf.keras.layers.Dense(1)  # Sin activación = regresión
])

# ARQUITECTURA:
# Input: secuencia de 56 días → SimpleRNN(32 neuronas) → Dense(1) → Predicción
#
# La capa SimpleRNN devuelve solo el último estado oculto (32 valores)
# La capa Dense convierte esos 32 valores en 1 predicción

In [ ]:
# Entrenamos el modelo con mayor learning rate (0.05 vs 0.02)
# Más neuronas permiten un learning rate más alto sin divergir
fit_and_evaluate(univar_model, train_ds, valid_ds, learning_rate=0.05)

Esto ya es otra cosa. Mejor que una capa con una sola neurona, veamos ahora (bueno en la siguiente sesión) como funciona con varias capas recurrentes.

***

### Deep RNNs

Hora de aplicar unas cuantas capas de recurrentes a ver si captan más patrones temporales y mejora nuestro regresor.

In [ ]:
# ============================================================================
# DEEP RNN: Múltiples capas recurrentes apiladas
# ============================================================================

tf.random.set_seed(42)

deep_model = tf.keras.Sequential([
    # Primera capa RNN: return_sequences=True
    # Esto hace que devuelva TODOS los estados ocultos, no solo el último
    # Salida: (batch, 56, 32) - para cada uno de los 56 pasos, 32 valores
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),
    
    # Segunda capa RNN: también devuelve todas las secuencias
    # Entrada: (batch, 56, 32) → Salida: (batch, 56, 32)
    tf.keras.layers.SimpleRNN(32, return_sequences=True),
    
    # Tercera capa RNN: return_sequences=False (por defecto)
    # Solo devuelve el último estado oculto
    # Entrada: (batch, 56, 32) → Salida: (batch, 32)
    tf.keras.layers.SimpleRNN(32),
    
    # Capa de salida
    # Entrada: (batch, 32) → Salida: (batch, 1)
    tf.keras.layers.Dense(1)
])

# ¿POR QUÉ return_sequences=True?
# Cada capa RNN necesita una secuencia como entrada (excepto la última)
# Si usáramos False en las primeras capas, solo pasaríamos 32 valores
# a la siguiente capa, perdiendo la información temporal
#
# SOLO en la última capa RNN usamos return_sequences=False
# porque la capa Dense no necesita secuencias, solo un vector

In [ ]:
# Entrenamos la Deep RNN
# Learning rate más bajo (0.01) porque el modelo es más profundo
# Modelos más profundos necesitan learning rates más conservadores
fit_and_evaluate(deep_model, train_ds, valid_ds, learning_rate=0.01)

### Series multivariantes

Pues como tenemos más series temporales que covarian con la de "rail", vamos a usarlas para hacer un modelo multivariante, parecido a usar ARIMAX o SARIMAX (aunque más potente)

In [ ]:
# ============================================================================
# PREPARACIÓN DE DATOS MULTIVARIANTES
# ============================================================================

# Ahora usaremos MÚLTIPLES series temporales como entrada
# No solo 'rail', también 'bus' y el tipo de día siguiente

# Seleccionamos las columnas 'bus' y 'rail' y normalizamos
df_mulvar = df[["bus", "rail"]] / 1e6

# FEATURE ENGINEERING: Añadimos el tipo de día SIGUIENTE
# shift(-1) desplaza la columna hacia arriba (miramos 1 día al futuro)
# Esto es información valiosa: saber si mañana es laboral/fin de semana/festivo
df_mulvar["next_day_type"] = df["day_type"].shift(-1)

# Convertimos la variable categórica a variables dummy (one-hot encoding)
# 'day_type' tiene valores como 'W' (weekday), 'A' (saturday), 'U' (sunday/holiday)
# get_dummies crea columnas binarias: next_day_type_W, next_day_type_A, etc.
df_mulvar = pd.get_dummies(df_mulvar, dtype=float)  # float para compatibilidad con TF

# RESULTADO: Ahora tenemos 5 columnas en lugar de 2:
# - bus (normalizado)
# - rail (normalizado)  
# - next_day_type_W (1 si mañana es laboral, 0 si no)
# - next_day_type_A (1 si mañana es sábado, 0 si no)
# - next_day_type_U (1 si mañana es domingo/festivo, 0 si no)

In [ ]:
# Visualizamos las primeras filas del dataframe multivariante
df_mulvar.head(5)

In [ ]:
# Dividimos los datos multivariantes en train/valid/test
# Mismos periodos que antes
mulvar_train = df_mulvar["2016-01":"2019-05"]
mulvar_valid = df_mulvar["2019-04-06":"2019-06"]  # 56 días antes de junio
mulvar_test = df_mulvar["2019-07":]

In [ ]:
# ============================================================================
# CREACIÓN DE DATASETS MULTIVARIANTES
# ============================================================================

tf.random.set_seed(42)

# DATASET DE ENTRENAMIENTO
train_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),  # Usamos TODAS las 5 columnas como entrada
    targets=mulvar_train["rail"][seq_length:],  # Target: solo predecimos 'rail'
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)

# DATASET DE VALIDACIÓN  
valid_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

# IMPORTANTE: 
# - INPUT: 5 features (bus, rail, next_day_type_W, next_day_type_A, next_day_type_U)
# - OUTPUT: 1 target (rail)
# 
# Estamos usando información de 'bus' y tipo de día para mejorar
# la predicción de 'rail' (uso del tren/metro)

In [ ]:
# Contamos cuántos batches hay en el dataset de entrenamiento
print(len(list(train_mulvar_ds)))

Veamos que pinta tiene un batch cualquiera de los 38 batches

In [ ]:
# Visualizamos un batch aleatorio (el 23) para entender la estructura
list(train_mulvar_ds)[23]

# ESTRUCTURA:
# - Primer elemento: array de shape (batch_size, 56, 5)
#   → 32 secuencias, cada una con 56 pasos temporales, cada paso con 5 features
# - Segundo elemento: array de shape (batch_size,)
#   → 32 targets (un valor de 'rail' por cada secuencia)

Y el último

In [ ]:
# Visualizamos el último batch del dataset
# Puede tener menos de 32 secuencias si no había suficientes datos
list(train_mulvar_ds)[-1]

In [ ]:
# ============================================================================
# MODELO MULTIVARIANTE: SimpleRNN con 5 features de entrada
# ============================================================================

tf.random.set_seed(42)

mulvar_model = tf.keras.Sequential([
    # IMPORTANTE: input_shape=[None, 5]
    # - None: longitud de secuencia variable
    # - 5: ahora tenemos 5 características por paso temporal (antes era 1)
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    
    tf.keras.layers.Dense(1)  # Salida: una predicción
])

# VENTAJA del modelo multivariante:
# Puede aprender correlaciones entre las variables
# Ej: Si el uso de bus baja, quizás el de tren sube
# Ej: Si mañana es fin de semana, el uso será diferente

In [ ]:
# Entrenamos y evaluamos el modelo multivariante
fit_and_evaluate(
    mulvar_model, 
    train_mulvar_ds, 
    valid_mulvar_ds,
    learning_rate=0.05
)

# Esperamos que mejore respecto al modelo univariante
# porque tiene más información disponible

Hmm, ha mejorado (cosa que no logramos con los SARIMAX en su día), pero siempre con predicción a un día. En la siguiente sesión ampliaremos el horizonte temporal y ya compararemos con nuestro baseline inicial.

***

## Predecir varios intervalos temporales en el futuro

Pero al igual que hicimos con el autoarima, nos gustaría predecir no sólo al día siguiente sino los x días siguientes...

**Caso 1**: Predecir tipo ARIMA (seq2vec)

Como hacemos con el predict(num_intervalos) del autoarima (vamos prediciendo añadiendo las predicciones de días futuros como elementos de la secuencia a predecir )

In [ ]:
# ============================================================================
# PREDICCIÓN SECUENCIAL: Método seq2vec (como ARIMA)
# ============================================================================

# Preparamos los datos de entrada: los primeros 56 días de validación
# np.newaxis añade dimensiones para que tenga shape (1, 56, 1)
# - 1er dimensión: batch size = 1
# - 2da dimensión: secuencia de 56 días
# - 3ra dimensión: 1 feature (solo 'rail')
X = rail_valid.to_numpy()[np.newaxis, :seq_length, np.newaxis]

In [ ]:
# Verificamos la forma del array de entrada
X.shape  # Debe ser (1, 56, 1)

In [ ]:
# ============================================================================
# PREDICCIÓN ITERATIVA: Predecir 14 días hacia el futuro
# ============================================================================

# MÉTODO: Predecir un día, añadirlo a la secuencia, predecir el siguiente
# Es como el método predict() de ARIMA

for step_ahead in range(14):
    # Predecimos el siguiente día basándonos en la secuencia actual
    y_pred_one = univar_model.predict(X)
    
    # Añadimos la predicción al final de la secuencia
    # reshape(1, 1, 1): convertimos el valor predicho al formato correcto
    # concatenate: pegamos la nueva predicción al final de X
    # axis=1: concatenamos a lo largo del eje temporal (la secuencia)
    X = np.concatenate([X, y_pred_one.reshape(1, 1, 1)], axis=1)
    
# RESULTADO: X ahora tiene 56 + 14 = 70 valores
# Los últimos 14 son nuestras predicciones

# DESVENTAJA: Los errores se acumulan. Si nos equivocamos en día 1,
# ese error afecta a la predicción del día 2, y así sucesivamente

In [ ]:
# ============================================================================
# VISUALIZACIÓN DE RESULTADOS: Predicción vs Realidad
# ============================================================================

# Extraemos las últimas 14 predicciones y creamos una Serie de pandas
Y_pred = pd.Series(
    X[0, -14:, 0],  # Últimos 14 valores de X
    index=pd.date_range("2019-06-01", "2019-06-14")  # Fechas de junio
)

# Creamos el gráfico
fig, ax = plt.subplots(figsize=(8, 3.5))

# Graficamos los valores reales (contexto + período a predecir)
(rail_valid * 1e6)["2019-04-06":"2019-06-14"].plot(
    label="True", 
    marker=".", 
    ax=ax
)

# Graficamos las predicciones (desnormalizando)
(Y_pred * 1e6).plot(
    label="Predictions", 
    grid=True, 
    marker="x", 
    color="r", 
    ax=ax
)

# Línea vertical marcando "hoy" (31 de mayo, último día de entrenamiento)
ax.vlines("2019-05-31", 0, 1e6, color="k", linestyle="--", label="Today")

# Ajustamos los límites del eje Y para ver mejor las predicciones
ax.set_ylim([200_000, 800_000])

plt.legend(loc="center left")
plt.show()

# INTERPRETACIÓN:
# - Azul: valores reales
# - Rojo: predicciones del modelo
# - Línea discontinua negra: marca el inicio de las predicciones

In [ ]:
# Extraemos los valores reales de junio 1-14 para comparar
y_valid = rail_valid["2019-06-01":"2019-06-14"]

In [ ]:
# ============================================================================
# EVALUACIÓN DE LA PREDICCIÓN A 14 DÍAS
# ============================================================================

# RMSE: Error cuadrático medio (penaliza más los errores grandes)
print("RMSE:", np.sqrt(mean_squared_error(y_valid, Y_pred)) * 1e06)

# MAPE: Error porcentual absoluto medio (fácil de interpretar)
print("MAPE:", mean_absolute_percentage_error(y_valid, Y_pred) * 100)

# AHORA SÍ podemos comparar con el baseline SARIMA
# Ambos predicen 14 días hacia el futuro

Hmmm, no está mal, ¿no?

**Caso 2**: Predecir seq2seq

Vamos a crear una red que entrene para predecir los 14 días siguientes de una vez.  

Para ello preparamos los datos de entrada de forma que el target serán ahora los 14 días siguientes a cada instante...   

Por tanto, si nuestras secuencias tienen 56 días, ahora el target serán 56 vectores de 14 valores (ejemplo 2 del surf, pero con 56 de tamaño de secuencia y 14 de predicción)

In [ ]:
# ============================================================================
# PREPARACIÓN DE DATOS PARA PREDICCIÓN SEQ2SEQ
# ============================================================================

tf.random.set_seed(42)

# Función auxiliar para dividir las secuencias en entrada y target
def split_inputs_and_targets(mulvar_series, ahead=14, target_col=1):
    """
    Divide cada secuencia en:
    - Input: todos los valores excepto los últimos 'ahead' días
    - Target: los últimos 'ahead' valores de la columna 'target_col'
    
    Args:
        mulvar_series: secuencia de shape (batch, seq_length, features)
        ahead: cuántos pasos predecir (default: 14)
        target_col: índice de la columna a predecir (1 = 'rail')
    
    Returns:
        (inputs, targets)
    """
    # [:, :-ahead]: todos los valores excepto los últimos 14
    # [:, -ahead:, target_col]: los últimos 14 valores de la columna 'rail'
    return mulvar_series[:, :-ahead], mulvar_series[:, -ahead:, target_col]

# DATASET DE ENTRENAMIENTO
ahead_train_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),
    targets=None,  # No especificamos targets aquí
    sequence_length=seq_length + 14,  # Secuencias más largas (56+14=70)
    batch_size=32,
    shuffle=True,
    seed=42
).map(split_inputs_and_targets)  # Aplicamos la función de división

# DATASET DE VALIDACIÓN
ahead_valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=None,
    sequence_length=seq_length + 14,
    batch_size=32
).map(split_inputs_and_targets)

# ESTRUCTURA FINAL:
# Input: (batch, 56, 5) - 56 días con 5 features cada uno
# Target: (batch, 14) - 14 valores de 'rail' a predecir

In [ ]:
# Visualizamos un batch de entrenamiento para entender la estructura
list(ahead_train_ds)[0]

# ESTRUCTURA:
# - Input: (32, 56, 5) - 32 secuencias de 56 días con 5 features
# - Target: (32, 14) - 32 secuencias de 14 predicciones cada una

In [ ]:
# Visualizamos el primer batch de validación
list(ahead_valid_ds)[0]

In [ ]:
# ============================================================================
# MODELO SEQ2SEQ: Predice 14 días directamente
# ============================================================================

tf.random.set_seed(42)

ahead_model = tf.keras.Sequential([
    # Capa recurrente: procesa la secuencia de 56 días
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    
    # Capa de salida: 14 neuronas (una por cada día a predecir)
    tf.keras.layers.Dense(14)
])

# DIFERENCIA con el método anterior:
# - Método anterior (seq2vec): predicción iterativa, 1 día a la vez
# - Este método (seq2seq): predice LOS 14 DÍAS DE UNA VEZ
#
# VENTAJAS:
# - Los errores no se acumulan tanto
# - El modelo aprende a predecir directamente 14 días
# - Más rápido en inferencia (una sola pasada)
#
# DESVENTAJAS:
# - Más difícil de entrenar
# - Necesita más datos de entrenamiento

In [ ]:
# Entrenamos el modelo seq2seq
fit_and_evaluate(
    ahead_model, 
    ahead_train_ds, 
    ahead_valid_ds,
    learning_rate=0.02
)

In [ ]:
# ============================================================================
# PREDICCIÓN CON EL MODELO SEQ2SEQ
# ============================================================================

# Preparamos la entrada: los primeros 56 días de validación con 5 features
X = mulvar_valid.to_numpy()[np.newaxis, :seq_length]  # shape [1, 56, 5]

# Predicción: una sola llamada para obtener los 14 días
Y_pred = ahead_model.predict(X)  # shape [1, 14]

Y_pred  # Visualizamos las predicciones

In [ ]:
# ============================================================================
# VISUALIZACIÓN SEQ2SEQ: Predicción vs Realidad
# ============================================================================

# Convertimos las predicciones a una Serie de pandas
Y_pred = pd.Series(
    Y_pred[0],  # Extraemos el array de 14 predicciones
    index=pd.date_range("2019-06-01", "2019-06-14")
)

# Creamos el gráfico
fig, ax = plt.subplots(figsize=(8, 3.5))

# Valores reales (desnormalizados)
(rail_valid * 1e6)["2019-04-06":"2019-06-14"].plot(
    label="True", 
    marker=".", 
    ax=ax
)

# Predicciones (desnormalizadas)
(Y_pred * 1e6).plot(
    label="Predictions", 
    grid=True, 
    marker="x", 
    color="r", 
    ax=ax
)

# Marcamos el día de "hoy" (fin del período de entrenamiento)
ax.vlines("2019-05-31", 0, 1e6, color="k", linestyle="--", label="Today")
ax.set_ylim([200_000, 800_000])

plt.legend(loc="center left")
plt.show()

In [ ]:
# ============================================================================
# EVALUACIÓN DEL MODELO SEQ2SEQ
# ============================================================================

print("RMSE:", np.sqrt(mean_squared_error(y_valid, Y_pred)) * 1e06)
print("MAPE:", mean_absolute_percentage_error(y_valid, Y_pred) * 100)

# Compara estos resultados con:
# 1. El baseline SARIMA
# 2. El método seq2vec anterior
# ¿Cuál funciona mejor?

Una mejora sobre el baseline aunque no sobre la predicción sobre la predicción. La mejora puede ser debida a la predicción a 14 días entrenando con secuencias de targets o bien a que hemos empleado el modelo de series multivariante. Te dejo como ejercicio el que pruebes con el modelo univariante y predicción a 14 días.b

***

Las celdas sencillas recurrentes no son las más utilizadas hoy en día, sino que existen otros dos tipos de celdas/neuronas recurrentes que buscan mejorar ese mecanismo de hipotética memoria.   

Estas dos celdas son la LSTM (Long-short term memory) que intenta regular el impacto de periodos o elementos de la secuencia más lejanos y de los más cercanos al punto tratado. Por otro lado la celda GRU (Gated Recurrent Unit), simplifica la anterior pero también intentando regular el impacto de los diferentes elementos de la secuencia.

# LSTMs

Las celdas LSTM (Long-Short Term Memory), buscan aumentar la capacidad de "memoria". Para ello ahora además de un hidden_state, devuelven un c_state en lo que vendría a ser hidden_state -> memoria a corto plazo, c_state -> memoria a largo plazo.


Por otro lado, las celdas LSTM se pueden incorpar como tal o usando una capa especial LSTM.

In [ ]:
# ============================================================================
# LSTM: Long Short-Term Memory
# ============================================================================

tf.random.set_seed(42)

lstm_model = tf.keras.models.Sequential([
    # Capa LSTM con 32 unidades
    # LSTM es más sofisticada que SimpleRNN
    tf.keras.layers.LSTM(32, input_shape=[None, 5]),
    
    # Capa de salida: 14 predicciones
    tf.keras.layers.Dense(14)
])

# ¿QUÉ ES LSTM?
# Long Short-Term Memory - Memoria a Corto y Largo Plazo
#
# PROBLEMA DE SimpleRNN:
# - Olvida información antigua (gradient vanishing)
# - Difícil aprender dependencias a largo plazo
#
# SOLUCIÓN DE LSTM:
# - Tiene "puertas" (gates) que controlan qué información mantener/olvidar
# - 3 puertas principales:
#   1. Forget gate: decide qué olvidar del estado anterior
#   2. Input gate: decide qué nueva información añadir
#   3. Output gate: decide qué información sacar
#
# - Mantiene 2 estados:
#   1. Cell state (c): memoria a largo plazo
#   2. Hidden state (h): memoria a corto plazo
#
# VENTAJAS:
# - Mejor para secuencias largas
# - Captura dependencias a largo plazo
# - Menos problemas de gradient vanishing
#
# DESVENTAJAS:
# - Más parámetros (más lento de entrenar)
# - Más complejo de entender

In [ ]:
# Entrenamos el modelo LSTM
# Learning rate más alto (0.1) porque LSTM es más estable
fit_and_evaluate(
    lstm_model, 
    ahead_train_ds, 
    ahead_valid_ds,
    learning_rate=0.1, 
    epochs=500
)

# NOTA: LSTM suele converger más rápido que SimpleRNN

In [ ]:
# Hacemos predicciones con el modelo LSTM
X = mulvar_valid.to_numpy()[np.newaxis, :seq_length]  # shape [1, 56, 5]
Y_pred = lstm_model.predict(X)  # shape [1, 14]
Y_pred

In [ ]:
# ============================================================================
# VISUALIZACIÓN LSTM: Predicción vs Realidad
# ============================================================================

Y_pred = pd.Series(
    Y_pred[0],
    index=pd.date_range("2019-06-01", "2019-06-14")
)

fig, ax = plt.subplots(figsize=(8, 3.5))

# Valores reales
(rail_valid * 1e6)["2019-04-06":"2019-06-14"].plot(
    label="True", 
    marker=".", 
    ax=ax
)

# Predicciones LSTM
(Y_pred * 1e6).plot(
    label="Predictions", 
    grid=True, 
    marker="x", 
    color="r", 
    ax=ax
)

ax.vlines("2019-05-31", 0, 1e6, color="k", linestyle="--", label="Today")
ax.set_ylim([200_000, 800_000])

plt.legend(loc="center left")
plt.show()

In [ ]:
# ============================================================================
# EVALUACIÓN DEL MODELO LSTM
# ============================================================================

print("RMSE:", np.sqrt(mean_squared_error(y_valid, Y_pred)) * 1e06)
print("MAPE:", mean_absolute_percentage_error(y_valid, Y_pred) * 100)

# ¿Mejora LSTM respecto a SimpleRNN?
# Compara con los resultados anteriores

# GRUs





Las LSTM y las GRU son mucho más efectivas que las simples RNN y son las que se usan hoy en día. Como las LSTM, las GRU tienen su propia capa aunque puedes utilizarlas como celdas en una RNN layer (no en una SimpleRNN) y el funcionamiento es similar.


In [ ]:
# ============================================================================
# GRU: Gated Recurrent Unit
# ============================================================================

tf.random.set_seed(42)

gru_model = tf.keras.Sequential([
    # Capa GRU con 32 unidades
    # GRU es una simplificación de LSTM
    tf.keras.layers.GRU(32, input_shape=[None, 1]),
    
    # Capa de salida
    tf.keras.layers.Dense(1)
])

# ¿QUÉ ES GRU?
# Gated Recurrent Unit - Unidad Recurrente con Puertas
#
# RELACIÓN CON LSTM:
# - Versión simplificada de LSTM
# - Solo 2 puertas (vs 3 en LSTM):
#   1. Reset gate: decide qué olvidar del pasado
#   2. Update gate: decide cuánta información nueva añadir
#
# - Solo 1 estado (vs 2 en LSTM):
#   - Hidden state (h) que combina memoria corto y largo plazo
#
# VENTAJAS vs LSTM:
# - Menos parámetros (más rápido de entrenar)
# - Más fácil de entrenar
# - Generalmente similar rendimiento a LSTM
# - Mejor para datasets pequeños
#
# VENTAJAS vs SimpleRNN:
# - Mejor memoria a largo plazo
# - Menos gradient vanishing
#
# CUÁNDO USAR:
# - Prueba GRU primero (más rápido)
# - Si no funciona bien, prueba LSTM (más potente)
# - SimpleRNN solo para casos muy simples

In [ ]:
# Entrenamos el modelo GRU
# Usamos el dataset univariante (solo 'rail')
fit_and_evaluate(
    gru_model, 
    train_ds, 
    valid_ds, 
    learning_rate=0.1, 
    epochs=500
)

# NOTA: Comparamos GRU univariante para ver si con menos datos
# puede competir con los modelos multivariantes

In [ ]:
# ============================================================================
# PREDICCIÓN ITERATIVA CON GRU
# ============================================================================

# Preparamos la entrada inicial
X = rail_valid.to_numpy()[np.newaxis, :seq_length, np.newaxis]

In [ ]:
# Predecimos 14 días de forma iterativa (como con SimpleRNN)
for step_ahead in range(14):
    y_pred_one = gru_model.predict(X)
    X = np.concatenate([X, y_pred_one.reshape(1, 1, 1)], axis=1)

In [ ]:
# ============================================================================
# VISUALIZACIÓN GRU: Predicción vs Realidad
# ============================================================================

Y_pred = pd.Series(
    X[0, -14:, 0],
    index=pd.date_range("2019-06-01", "2019-06-14")
)

fig, ax = plt.subplots(figsize=(8, 3.5))

# Valores reales
(rail_valid * 1e6)["2019-04-06":"2019-06-14"].plot(
    label="True", 
    marker=".", 
    ax=ax
)

# Predicciones GRU
(Y_pred * 1e6).plot(
    label="Predictions", 
    grid=True, 
    marker="x", 
    color="r", 
    ax=ax
)

ax.vlines("2019-05-31", 0, 1e6, color="k", linestyle="--", label="Today")
ax.set_ylim([200_000, 800_000])

plt.legend(loc="center left")
plt.show()

In [ ]:
# ============================================================================
# EVALUACIÓN FINAL: GRU
# ============================================================================

print("RMSE:", np.sqrt(mean_squared_error(y_valid, Y_pred)) * 1e06)
print("MAPE:", mean_absolute_percentage_error(y_valid, Y_pred) * 100)

# COMPARACIÓN FINAL:
# ------------------
# Ahora tienes resultados de:
# 1. SARIMA (baseline estadístico)
# 2. SimpleRNN (red recurrente básica)
# 3. Deep SimpleRNN (múltiples capas)
# 4. LSTM (memoria a largo plazo)
# 5. GRU (versión simplificada de LSTM)
#
# PREGUNTAS PARA REFLEXIONAR:
# - ¿Cuál modelo tiene mejor RMSE?
# - ¿Cuál modelo tiene mejor MAPE?
# - ¿Vale la pena la complejidad extra de LSTM/GRU?
# - ¿Los modelos multivariantes mejoran significativamente?
# - ¿El método seq2seq es mejor que la predicción iterativa?
#
# CONCLUSIONES TÍPICAS:
# - GRU y LSTM suelen ser mejores que SimpleRNN
# - Modelos multivariantes suelen superar a univariantes
# - La predicción iterativa acumula errores
# - El método seq2seq es más estable pero más difícil de entrenar
# - A veces, un buen SARIMA puede competir con redes neuronales simples